## CUQIpy component final project

**Note: under construction to be finalized by midnight 21 July 2024** 

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cuqi
from cuqi.model import Model #, LinearModel
from cuqi.geometry import MappedGeometry, Continuous1D, KLExpansion #, Geometry, Continuous
from cuqi.distribution import Gaussian
from scipy.linalg import lu_factor, lu_solve

from testBeam import BeamModel1D

In [10]:
class hydraulic_class():
    def __init__(self, N, L=1, source_idx=0):
        self.L = L
        self.N = N
        self.x = np.linspace(self.L/self.N,1,self.N)
        self.dx = self.L/self.N
        self.source_idx = source_idx
        self.source()

    def forward(self, a):
        diag1 = -(a[1:] + a[:-1])
        diag1 = np.concatenate([diag1,[-a[-1]]])
        diag2 = a[1:]

        Dxx = np.diag(diag1) + np.diag(diag2,-1) + np.diag(diag2,1)
        Dxx /= self.dx*self.dx

        lu, piv = lu_factor(Dxx)

        sol = lu_solve((lu, piv), self.b_terms) 

        return np.array(sol)

    def source(self, n_source=5, std=0.02):
        dist = self.L/(n_source+1)
        source_coords = np.linspace( dist,self.L-dist, n_source )
        self.b_terms =  np.exp( -0.5*(self.x - source_coords[self.source_idx])**2/std/std )/std/np.sqrt(2*np.pi) 


- Create a mapped KL expansion domain geometry with the mapping `lambda x: exp(x)`.
- Create a continuous range geometry for the range.
- Create a user define CUQIpy model representing the forward model (hydraulic).
- Create a Gaussian i.i.d. prior
- Create a Gaussian likelihood (for one injection pattern)
- Create a synthetic data from the given true porosity.
- Create a posterior
- Use MH for sampling
- Use NUTS for sampling
- Compare the results (ess, pair_plot, trace_plot, plot_ci)
- Comment on the computational cost of nuts (tree size, FD)
- Create a posterior with multiple likelihoods (for multiple injection patterns)
- Use MH for sampling
- Comment on how adding more data helped the inference.



In [36]:
# Data and initialization
U = lambda x: np.exp(x)             # Define mapping
L = 1                               # domain length 1
N_hydro = 128                       # Number of measurement points for hydrolic problem
N_points = 10                       # Number of discretization points
N_modes = 5                         # Number of terms in KL expansion
grid1 = np.linspace(0,L,N_points)   # Discretization for domain

# Create Beam and hydrolic models
model_hydrolic = Model(forward=hydraulic_class(N=N_points, L=L).forward, 
                range_geometry=Continuous1D(grid=grid1),
                domain_geometry=MappedGeometry(KLExpansion( grid=grid1,num_modes=N_modes ), map = U))

model_Beam = Model(forward=BeamModel1D(nelx=N_points, Lx=L).forward, 
                range_geometry=Continuous1D(grid=grid1),
                domain_geometry=MappedGeometry(KLExpansion( grid=grid1,num_modes=N_modes ), map = U))

# # Test both mdoels for som input
# print(model_hydrolic(np.ones(5)))
# print(model_Beam(np.ones(5)))

# # Gaussian prior -> test it on beam domain
# x = Gaussian(0,1,geometry=model_Beam.domain_geometry);
# x.sample(100).plot();

# Gaussian Likelihood
data_test = np.ones(5)
likelihood = cuqi.distribution.Gaussian(mean=model_Beam, cov=0.05**2).to_likelihood(data_test)
print(likelihood)

# Create sunthetic data


CUQI Gaussian Likelihood function. Parameters ['thkVec'].
